In [1]:
import cv2
import numpy as np

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [2]:
# Initialize Detector & load pre-trained weight
prototxt_path = "./caffe_dnn_module/deploy.prototxt.txt"
weights_path = "./caffe_dnn_module/res10_300x300_ssd_iter_140000.caffemodel"

detector = cv2.dnn.readNet(prototxt_path, weights_path)

# Load pre-trained CNN model
CNN = load_model("./checkpoint/AbridgedMobilenet")


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x000002EE74A47388> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002EE749F9788>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002EE749F9788> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002EE749D4648>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002EE749D4648> and <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002EE74817248>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002EE74820088> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002EE7481D348>).

Two checkpoint references resolved to different objects (<tensor

In [3]:
# Load test image
test_img = cv2.imread("./data/data2/with_mask/with_mask_734.jpg")


In [4]:
# Convert image to blob in Caffe Framework
blob = cv2.dnn.blobFromImage(test_img, 1.0, (300, 300),
                             (104.0, 177.0, 123.0))
(h, w) = test_img.shape[:2]

# Face Detection
detector.setInput(blob)
detections = detector.forward()
print(max(detections[0,0,:,2]))

0.89163685


In [5]:
# Threshold for detecting faces
threshold = 0.2

for i in range(0, detections.shape[2]):
    # Confidence of bounding box
    confidence = detections[0, 0, i, 2]

    if confidence > threshold:
        # Get minX, minY, maxX, maxY of bounding box
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        print(confidence, startX, startY, endX, endY)

        text = "{:2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10

        # Draw Bounding Box
        cv2.rectangle(test_img, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # Print Confidence
        cv2.putText(test_img, text, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imshow("result", test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



0.89163685 465 557 1139 1294
0.6782107 146 240 790 931
0.3305762 2166 2235 2869 3012
0.283053 2169 3121 2870 3871
